**1. Parsing the System of Equations**

In [50]:
import pathlib

def load_system(path: pathlib.Path) -> tuple[list[list[float]], list[float]]:
    fd = path.open()
    A = []
    B = []
    for line in fd:
        x,op1,y,op2,z,_,b = line.split()
        B.append(float(b))
        
        x1 = x.rstrip('x')
        y = op1+y
        y1 = y.rstrip('y')
        z = op2+z
        z1 = z.rstrip('z')
        
        match x1:
            case '':
                x = 1.0
            case '-':
                x = -1.0
            case _:
                x = float(x1)
        match y1:
            case '+':
                y = 1.0
            case '-':
                y = -1.0
            case _:
                y = float(y1)
        match z1:
            case '+':
                z = 1.0
            case '-':
                z = -1.0
            case _:
                z = float(z1)
        A.append([x,y,z])
                
    return A,B
    
    

load_system(pathlib.Path("system.txt"))
A, B = load_system(pathlib.Path("system.txt"))
print(f"{A=} {B=}")

A=[[2.0, 3.0, -1.0], [1.0, -1.0, 4.0], [3.0, 1.0, 2.0]] B=[5.0, 6.0, 7.0]


**2 Matrix and Vector Operations**

*1. Determinant:*

In [65]:
def determinant(matrix: list[list[float]]) -> float:
    return matrix[0][0] * (matrix[1][1] * matrix[2][2] - matrix[1][2] * matrix[2][1]) - matrix[0][1] * (matrix[1][0] * matrix[2][2] - matrix[1][2] * matrix[2][0]) + matrix[0][2] * (matrix[1][0] * matrix[2][1] - matrix[1][1] * matrix [2][0])

print(f"{determinant(A)=}")

determinant(A)=13.999999999999986


*2. Trace:*

In [4]:
def trace(matrix: list[list[float]]) -> float:
    return matrix[0][0] + matrix[1][1] + matrix[2][2]

print(f"{trace(A)=}")

trace(A)=3.0


*3. Vector norm:*

In [5]:
import math
def norm(vector: list[float]) -> float:
    return math.sqrt(vector[0] ** 2 + vector[1] ** 2 + vector[2] ** 2)

print(f"{norm(B)=}")

norm(B)=10.488088481701515


*4. Transpose of matrix:*

In [6]:
def transpose(matrix):

    transposed_matrix = [[0] * 3 for _ in range(3)]
    
    for i in range(3):
        for j in range(3):
            transposed_matrix[j][i] = matrix[i][j]
    return transposed_matrix


print(f"{transpose(A)=}")

transpose(A)=[[2.0, 1.0, 3.0], [3.0, -1.0, 1.0], [-1.0, 4.0, 2.0]]


*5. Matrix-vector multiplication:*

In [7]:
def multiply(matrix: list[list[float]], vector: list[float]) -> list[float]:
    result = [0, 0, 0]
    
    for line in range(0, 3):
        for col in range(0, 3):
            result[line] += matrix[line][col] * vector[col] 
    return result

print(f"{multiply(A, B)=}")

multiply(A, B)=[21.0, 27.0, 35.0]


**3 Solving using Cramer’s Rule**

In [8]:
def solve_cramer(matrix: list[list[float]], vector: list[float]) -> list[float]:
    Ax = [ [B[i]] + row[1:] for i,row in enumerate(matrix) ]
    Ay = [ [row[0]] + [B[i]] + [row[2]] for i,row in enumerate(matrix) ]
    Az = [ row[:2] + [B[i]] for i,row in enumerate(matrix) ]
    return [determinant(Ax)/determinant(A),determinant(Ay)/determinant(A),determinant(Az)/determinant(A)]
          

print(f"{solve_cramer(A, B)=}")

solve_cramer(A, B)=[0.35714285714285715, 2.0714285714285716, 1.9285714285714286]


**4. Solving using Inversion**

In [67]:
def determinant_minor(matrix: list[list[float]]) -> float:
    return matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]

def minor(matrix: list[list[float]], i: int, j: int) -> list[list[float]]:
    mminor = []
    for index,line in enumerate(matrix):
        if index == i:
            continue
        else:
            newline = []
            for column,value in enumerate(line):   
                if column != j:
                    newline.append(value)
            mminor.append(newline)
    return mminor

def cofactor(matrix: list[list[float]]) -> list[list[float]]:
    cofactor_matrix = []
    for i in range(len(matrix)):
        newline = []
        for j in range(len(matrix)):
            newline.append(((-1) **(i+j)) * determinant_minor(minor(matrix,i,j)))
        cofactor_matrix.append(newline)    
    return cofactor_matrix


def adjoint(matrix: list[list[float]]) -> list[list[float]]:
    return transpose(cofactor(matrix))

def inversion(matrix: list[list[float]]) -> list[list[float]]:
    adj_matrix = adjoint(matrix)
    x = 1 / determinant(matrix)
    inv_matrix = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
    for i,line in enumerate(matrix):
        for j in range(len(line)):
            inv_matrix[i][j] = x * adj_matrix[i][j]
    return inv_matrix

def solve(matrix: list[list[float]], vector: list[float]) -> list[float]:
    return multiply(inversion(matrix),vector)

print(f"{solve(A, B)=}")


solve(A, B)=[0.35714285714285987, 2.0714285714285734, 1.928571428571429]
